## Skab database

In [14]:

import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()


c.execute("""CREATE TABLE IF NOT EXISTS articles (
            headline text,  
            URL text UNIQUE,
            date text,
            is_nyhed integer,
            type_article text,
            text_body text,
            countries text,
            KM_index integer, 
            language text,
            media text,
            processed integer
            )""")


conn.commit()
conn.close()



## Hjælpefunktioner

In [ ]:
# GET COLUMN NAMES

conn = sqlite3.connect('database.db')
c = conn.execute('select * from articles')

# Fetch and print column names
column_names = [description[0] for description in c.description]
print("Column Names:", column_names)

conn.close()

Column Names: ['headline', 'URL', 'date', 'is_nyhed', 'type_article', 'text_body', 'countries', 'KM_index', 'language', 'media', 'processed']


In [ ]:
# GET 10 ITEMS
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.execute('SELECT headline, countries, processed FROM articles WHERE media = "DR"')
rows = c.fetchmany(5)
for row in rows:
    print(row)

conn.close()

('Store finansielle institutioner advarer om AI-boble. Er den ved at briste?', 'China, Denmark, United States of America', 1)
('Det er tid til at gøre op med rosenkålstraumet: Den smager ikke som for 40 år siden', 'Denmark', 1)
('Suveræn Pogacar vinder Lombardiet Rundt for femte år i træk', 'Belgium, Poland, Slovenia', 1)
('Aarhus-borgmester har succes med at være gakket på TikTok', 'China, Denmark, United States of America', 1)
('Fra vælgermøder til virale videoer: Lokalpolitikere danser på TikTok for at nå de unge vælgere', 'China', 1)


In [ ]:
# DELETE ALL DATA
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()

# Delete all rows from the table
c.execute('DELETE FROM articles')

# Commit the changes
conn.commit()
conn.close()

In [2]:
#  NUMBER OF ITEMS
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()

# Delete all rows from the table
c.execute('SELECT * FROM articles')
print(len(c.fetchall()))

# Commit the changes
conn.commit()
conn.close()

118


## Find unikke lande i artiklen

In [ ]:

import sqlite3
import analyze


conn = sqlite3.connect('database.db')
# Set row_factory to access columns by name, which is easier to read
conn.row_factory = sqlite3.Row 
c = conn.cursor()


# We only need the URL (as a unique ID) and the text_body for analysis
c.execute('SELECT URL, text_body FROM articles WHERE media = "DR" AND processed = 0')
articles_to_process = c.fetchall()

   
for article in articles_to_process:
    url = article['URL']
    text = article['text_body']

        
    countries_found = analyze.find_countries(text, analyze.demonym.DANISH, "Danish")

    if countries_found:
        print(f"Updating {url} -> Countries: {countries_found}")

        # Update the 'countries' column and set 'processed' to 1 (True) for that article
        c.execute("""
            UPDATE articles 
            SET countries = ?, processed = 1 WHERE URL = ?""", 
            (countries_found, url))
        
    else:
        c.execute("UPDATE articles SET processed = 1 WHERE URL = ?", (url,))

conn.commit()
print("Database has been updated successfully.")
conn.close()
        

Database has been updated successfully.
